In [1]:
import json

import sys

sys.dont_write_bytecode = True

import numpy as np

import datetime

import random

import math

import core

In [19]:
base = "BTC"

#base = "ETH"

#base = "LTC"



quote = "USDT"

historymins = 60*24*30*4 #60*24*30*4

interval = 10

dtend = datetime.datetime.strptime('2018-04-24 00:00', '%Y-%m-%d %H:%M')

dtstart = dtend - datetime.timedelta(minutes=historymins)

inp = core.getPriceExchange_v1('binance', interval, base, quote, historymins, dtend)

uncertainty_margin = 0.001

In [15]:
dtit = dtstart
idx  = datetime.datetime.strftime(dtit, '%Y-%m-%dT%H:%M')

In [ ]:
print(inp)

In [9]:
def portfolioToChart_OHLC(portfolio, traces=[], file="./chart.html"):

    # this will use the portfolio and its _buy/_sell entries to generate an OHLC chart.

    import plotly

    import plotly.plotly as py

    import plotly.graph_objs as go



    opens = []

    highs = []

    lows = []

    closes = []
    
    Vols = []
    
    trades = []

    dates = []

    annons = []

    
    for key, obj in portfolio.items():

        if key == "_": continue

        # dt = datetime.datetime.strptime(key, '%Y-%m-%d %H:%M') # if you use this, then plotly will display local dates -- but you must parse it everywhere below as well !

        dt = key

        opens.append(obj['open'])

        highs.append(obj['high'])

        lows.append(obj['low'])

        closes.append(obj['close'])
        
        Vols.append(obj['volume'])
        
        trades.append(obj['trades'])

        dates.append( dt )
       

    data = []

    trace = go.Candlestick(x=dates,

                        open=opens,

                        high=highs,

                        low=lows,

                        close=closes,

                        opacity=0.5)

    data.append(trace)


    for t in traces:

        trace = go.Scatter(

            name = t['name'],

            x = t['x'],

            y = t['y'],

            mode = 'lines', # +markers

            yaxis = t['yaxis'],

        )

        data.append(trace)
       

    layout = go.Layout(

        xaxis = dict(

            rangeslider = dict(

                visible = False

            ),

            type = "date",

        ),

        annotations = annons,

       # dragmode = "pan"

    )

    
    fig = go.Figure(data=data,layout=layout)
    config = {'scrollZoom': True}
    plotly.offline.plot(fig, config=config, filename=file)


In [20]:
portfolioToChart_OHLC(inp, traces=[], file="./chart.html")

In [21]:
import pandas as pd
import collections

In [22]:
#Code adopted from Ilya nevolin
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
# flatten all beyond first level:
for key in inp:
    inp[key] = flatten(inp[key])

# bamboo & pandas
df = pd.DataFrame(inp, columns=inp.keys())
df_T = df.T
df_T = df_T.sort_index(ascending=False,inplace=False)
df_T[:3]

In [69]:
def tech_indicators(data,ntime_per,m,n):
    
    h = data['high']

    l = data['low']
    
    o = data['open']

    c = data['close']
    
    Vol = data['volume']

    TP = (h+l+c)/3
    
# Commodity Channel Index 
    CCI = pd.Series((TP - TP.rolling(window=ntime_per,center=False).mean()) / (0.015 * TP.rolling(window=ntime_per,center=False).std()),
    name = 'CCI')
    
    data = data.join(CCI) 

# Ease of Movement 
    dm = ((h + l)/2) - ((h.shift(1) + l.shift(1))/2)

    br = (Vol/ 100000000) / ((h - l))
    
    EVM = dm / br 
    
    EVM_MA = pd.Series(EVM.rolling(window=ntime_per,center=False).mean(), name= 'EVM')
    
    data = data.join(EVM_MA)
    
# Rate of Change (ROC)

    N = c.diff(m)

    D = c.shift(m)

    ROC = pd.Series(N/D,name='Rate of Change')

    data = data.join(ROC)

# Force Index 

    FI = pd.Series(c.diff(ntime_per) * Vol, name = 'ForceIndex') 
    data = data.join(FI) 
    
# Golden Cross
    
    SMA = pd.Series(TP.rolling(window=m,center=False).mean(), name= 'SMA')
    LMA = pd.Series(TP.rolling(window=n,center=False).mean(), name= 'LMA')
    SMA2 = pd.Series(TP.rolling(window=m,center=False).mean().shift(1), name= 'SMA2')
    LMA2 = pd.Series(TP.rolling(window=n,center=False).mean().shift(1), name= 'LMA2')
    
    data = data.join(SMA)
    data = data.join(LMA)
    data = data.join(SMA2)
    data = data.join(LMA2)
    data = data.iloc[n:]
    
    return(data)

    

In [ ]:
CCI(df_T, 10)

In [26]:
# Ease of Movement 
def EVM(data, ntime_per): 
    
    h = data['high']

    l = data['low']

   

    dm = ((h + l)/2) - ((h.shift(1) + l.shift(1))/2)

    br = (Vol/ 100000000) / ((h - l))
    
    EVM = dm / br 
    
    EVM_MA = pd.Series(EVM.rolling(window=ntime_per,center=False).mean(), name= 'EVM')
    
    data = data.join(EVM_MA) 
    
    return data 

In [ ]:
EVM(SMA

In [44]:
# Golden Cross
def Golden_Cross(data,m,n):
    
    h = data['high']

    l = data['low']

    c = data['close']

    TP = (h+l+c)/3
    
    SMA = pd.Series(TP.rolling(window=m,center=False).mean(), name= 'SMA')
    LMA = pd.Series(TP.rolling(window=n,center=False).mean(), name= 'LMA')
    SMA2 = pd.Series(TP.rolling(window=m,center=False).mean().shift(1), name= 'SMA2')
    LMA2 = pd.Series(TP.rolling(window=n,center=False).mean().shift(1), name= 'LMA2')
    
    data = data.join(SMA)
    data = data.join(LMA)
    data = data.join(SMA2)
    data = data.join(LMA2)
    data = data.iloc[n:]
    
    return(data)

In [ ]:
Golden_Cross(df_T,10,30)

In [ ]:
tech_indicators(df_T,10,18,144)

In [71]:
data = tech_indicators(df_T,10,6,18)
data['Buy'] = data.apply(lambda x: 1 if x['SMA'] > x['LMA'] and x['SMA2'] < x['LMA2'] else 0,axis =1)
data['Sell'] = data.apply(lambda y: -1 if y['SMA'] < y['LMA'] and y['SMA2'] > y['LMA2'] else 0,axis =1)
data['Signal'] = data['Buy'] + data['Sell'] 

In [73]:
print(data.head())

                    close  count     high      low     open  trades  \
2018-04-23T21:00  8914.42   10.0  8924.11  8912.00  8924.11   974.0   
2018-04-23T20:50  8924.11   10.0  8928.00  8904.01  8912.00  1135.0   
2018-04-23T20:40  8909.02   10.0  8920.00  8889.97  8915.00  1108.0   
2018-04-23T20:30  8915.00   10.0  8915.00  8867.01  8868.00  1359.0   
2018-04-23T20:20  8868.00   10.0  8880.00  8858.00  8869.27  1129.0   

                      volume         CCI           EVM  Rate of Change  \
2018-04-23T21:00  124.567874  -50.989764 -2.253161e+07       -0.002750   
2018-04-23T20:50  174.239904  -26.356408 -2.560213e+07        0.000236   
2018-04-23T20:40  149.788528 -102.948604 -3.260356e+07       -0.001119   
2018-04-23T20:30  154.731979 -117.682306 -8.492977e+07        0.000944   
2018-04-23T20:20  109.915304 -171.397177 -1.313850e+08       -0.006310   

                   ForceIndex          SMA          LMA         SMA2  \
2018-04-23T21:00 -1827.410712  8917.348889  8922.334815 